In [2]:
import pandas, math
from sklearn.linear_model import LinearRegression, LogisticRegression
import sklearn.cross_validation as cv
import numpy as np

titanic = pandas.read_csv("train.csv")
titanic_test = pandas.read_csv('test.csv')

titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
titanic_test['Age'] = titanic_test['Age'].fillna(titanic['Age'].median())

# titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
# titanic.loc[titanic['Sex'] == 'female', 'Sex'] = 1
titanic.Sex.replace(['male', 'female'], [0, 1], inplace=True)
titanic_test.Sex.replace(['male', 'female'], [0, 1], inplace=True)
# titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0
# titanic_test.loc[titanic_test['Sex'] == 'female', 'Sex'] = 1


titanic.Embarked.fillna('S', inplace=True)
titanic_test.Embarked.fillna('S', inplace=True)

# titanic.loc[titanic.Embarked == 'S', 'Embarked'] = 0
# titanic.loc[titanic.Embarked == 'C', 'Embarked'] = 1
# titanic.loc[titanic.Embarked == 'Q', 'Embarked'] = 2
titanic['Embarked'].replace(['S', 'C', 'Q'], [0, 1, 2], inplace=True)
titanic_test['Embarked'].replace(['S', 'C', 'Q'], [0, 1, 2], inplace=True)
# titanic_test.loc[titanic_test.Embarked == 'S', 'Embarked'] = 0
# titanic_test.loc[titanic_test.Embarked == 'C', 'Embarked'] = 1
# titanic_test.loc[titanic_test.Embarked == 'Q', 'Embarked'] = 2

titanic_test['Fare'].fillna(titanic_test['Fare'].median(), inplace=True)

Testing and training data has been cleaned. Time to make some changes to the model.

In [3]:
titanic['LogAge'] = [math.log(val) for val in titanic['Age']]
titanic_test['LogAge'] = [math.log(val) for val in titanic_test['Age']]

titanic['Male'] = titanic.Sex
titanic['Female'] = titanic.Male
titanic['Female'].replace([0, 1], [0,1], inplace=True)

titanic_test['Male'] = titanic_test.Sex
titanic_test['Female'] = titanic_test.Male
titanic_test['Female'].replace([0, 1], [0,1], inplace=True)

Create a LogAge column which records the logarithm of the passenger's age, because a given change in age does not have the same effect across the entire distribution. 
Ideally, I would create a transformation that gave more resolution/weight to ages 0-20, and also 50-80. Differences in ages in the middle probably have less of an effect on survival.
Remove "ParCh" from the list of predictors.

In [4]:
predictors = ["Pclass", "Sex", "LogAge", "Fare", "SibSp", "Embarked"]
alg = LogisticRegression(random_state=1)
scores = cv.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)
print scores.mean()

0.802469135802


In [7]:
alg.fit(titanic[predictors], titanic["Survived"])
predictions = alg.predict(titanic_test[predictors])

submission = pandas.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })
    
submission.to_csv("kaggle.csv", index=False)

The new kaggle score is .77033